The author with the [most books](#3), the author with the [highest user rate](#4) and the [price-rating-review](#5) rates were compared. A few [visualization](#2) tools were used.

In addition, the useful data in the data set was shown by [selecting the feature](#7).

<font color = 'red'>
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#010501;
           font-size:200%;
           font-family:Cambria;
           letter-spacing:0.5px">

<b style="padding: 20px;
              color:white;
                  font-weight: bold;">Content:
</b>
</div>  

1. [Preparing Data](#1)
1. [Understanding the Data](#2)
    * [Top 20 Authors with Most Books on the List](#3)
    * [Top 20 Authors with Highest User Rating of Books](#4) 
    * [User Ratings, Prices and Reviews by Genre](#5)
    * [Visualization with violin plot](#6)    
1. [Importance Features Selection](#7)


In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import seaborn as sns  
import os
import warnings
warnings.filterwarnings("ignore")
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print("file path:",os.path.join(dirname, filename))

<a id = "1"></a><br>
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#0BB718;
           font-size:200%;
           font-family:Cambria;
           letter-spacing:0.5px">

<b style="padding: 20px;
              color:white;
                  font-weight: bold;">Preparing Data
</b>
</div> 


In [ ]:
load_data = pd.read_csv('../input/amazon-top-50-bestselling-books-2009-2019/bestsellers with categories.csv')
load_data.head(10).style.background_gradient(cmap='Greens')

In [ ]:
load_data.columns[load_data.isnull().any()]

In [ ]:
load_data.fillna(0)

<a id = "2"></a><br>
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#230FF0;
           font-size:200%;
           font-family:Cambria;
           letter-spacing:0.5px">

<b style="padding: 20px;
              color:white;
                  font-weight:bold;">Understanding the Data
</b>
</div> 

In [ ]:
load_data["User Rating"].max()

In [ ]:
df = load_data['Name'].nunique()
print("Author Numbers: ", df)

<a id = "3"></a><br>
* **Top 20 Authors with Most Books on the List.**

In [ ]:
dfHigh = load_data[['Author']]
f, ax = plt.subplots(figsize=(10,10))
dfHigh['Author'].value_counts()[:20].sort_values(ascending=True).plot(kind='barh', ax=ax, color = 'r')

<a id = "4"></a><br>
* **Top 20 Authors with Highest User Rating of Books**

In [ ]:
plt.figure(figsize=(15,15))
dff = load_data.groupby(["Author"]).sum()['User Rating']
RatingTop = dff.nlargest(20).plot(kind='pie', autopct='%1.3f%%') # three digits after the comma
plt.show() 

<a id = "5"></a><br>
* **User Ratings, Prices and Reviews by Genre**

* A [distplot](https://seaborn.pydata.org/generated/seaborn.displot.html) plots a univariate distribution of observations.
* The [FacetGrid](https://seaborn.pydata.org/generated/seaborn.FacetGrid.html) class can be used to individually visualize the distribution of a variable or the relationship between multiple variables within subsets of the data set.

In [ ]:
g = sns.FacetGrid(load_data, col = "Genre")
g.map(sns.distplot, "User Rating", bins = 25)
plt.show()

In [ ]:
g = sns.FacetGrid(load_data, col = "Genre")
g.map(sns.distplot, "Price", bins = 25)
plt.show()

In [ ]:
g = sns.FacetGrid(load_data, col = "Genre")
g.map(sns.distplot, "Reviews", bins = 25)
plt.show()

<a id = "6"></a><br>
* **Visualization with violin plot**

    > For violin plot visualization, the data needs to be standardized.
    A box plot only shows summary statistics such as mean/median and interquartile range, while the violin plot shows the full distribution of the data.
    
    >Centering and scaling happen independently on each feature by computing the relevant statistics on the samples in the training set. Mean and standard deviation are 
    then stored to be used on later data using transform.


    Standardization of a dataset is a common requirement for many machine learning estimators: they might behave badly if the individual features do not more or less 
    look like standard normally distributed data. For instance many elements used in the objective function of a learning 
    algorithm (such as the RBF kernel of Support Vector Machines or the L1 and L2 regularizers of linear models) assume that all features are centered around 0 and have 
    variance in the same order. If a feature has a variance that is orders of magnitude larger that others, it might dominate the objective function and make the 
    estimator unable to learn from other features correctly as expected.

In [ ]:
list = ['Name','Author','Genre']

data = load_data.drop(list, axis=1)
label = load_data.Genre
data.describe().style.background_gradient(cmap='Blues')

In [ ]:
stdData = (data - data.mean()) / (data.std()) # or we can use StandardScaler() 

stdData2 = pd.concat([label, stdData.iloc[:,:]], axis=1)
stdData2.head().style.background_gradient(cmap='Purples')

In [ ]:
stdData2 = pd.melt(stdData2, id_vars="Genre", var_name="features", value_name='value')

plt.figure(figsize=(15,10))
sns.violinplot(x="value", y="features", hue="Genre", data=stdData2, split=True, inner="quart")

In [ ]:
plt.figure(figsize=(10,10))
sns.boxplot(x="features", y="value", hue="Genre", data=stdData2)
plt.xticks(rotation=0)

In [ ]:
plt.figure(figsize=(15,10))
sns.swarmplot(x="value", y="features", hue="Genre", data=stdData2)

<a id = "7"></a><br>
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#0CF8FF;
           font-size:200%;
           font-family:Cambria;
           letter-spacing:0.5px">

<b style="padding: 20px;
              color:black;
                  font-weight:bold;">Importance Features Selection
</b>
</div>


* [Feature selection](https://scikit-learn.org/stable/modules/feature_selection.html) is to automatically select from the dataset the features that contribute the most to the prediction or output. Irrelevant features in the dataset can reduce the accuracy of many models.
* Tree-based estimators are used to calculate feature importance, eliminating unrelated features.
* ExtraTreesClassifier use all the data that reduces bias. That is, increasing data diversity and reducing bias improves model performance.

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

inValue = load_data.values

X = inValue[:,2:6]
Y = inValue[:,6]

model = ExtraTreesClassifier(n_estimators=4, random_state=42)
fit = model.fit(X, Y)

print("features importance: %s" % fit.feature_importances_)

genre = load_data.columns
pd.DataFrame({'names': genre[2:6], 'importance': fit.feature_importances_}).\
    sort_values(by='importance', ascending=False)

**I am waiting for your upvote, comments and advices**

# Thank you!